In [ ]:
import os

print(os.getcwd())


In [15]:
import asyncio
import numpy as np
import math
from TestClient import test_client

from sidecar import Sidecar
from ipyleaflet import *
from ipywidgets import Text

from colorhash import ColorHash

icon_size = [30, 30]
icon_anchor = [15, 15]

icon_dict = { 
    test_client.UserStatus.NON_GROUP_MEMBER : 
    Icon(icon_url='http://getdrawings.com/free-icon/man-icon-64.png', icon_size=icon_size, icon_anchor=icon_anchor),
    test_client.UserStatus.GROUP_MEMBER : 
    Icon(icon_url='http://getdrawings.com/free-icon/man-icon-58.png', icon_size=icon_size, icon_anchor=icon_anchor),
    test_client.UserStatus.GROUP_LEADER : 
    Icon(icon_url='http://getdrawings.com/free-icon/leadership-icon-png-62.png', icon_size=icon_size, icon_anchor=icon_anchor) 
}

bus_shared_route = [
    [37.453405, 126.953434], # 유전공학 가는 길
    [37.456974, 126.956029],
    [37.459593, 126.957169],
    [37.461191, 126.956311],
    [37.461714, 126.954569],
    [37.464158, 126.955256],
    [37.465682, 126.955031],
    [37.466176, 126.954098],
    [37.465695, 126.949492],
    [37.464605, 126.948730],
    [37.461666, 126.949359],
    [37.459477, 126.948694],
    [37.453847, 126.950207] # 신소재
]

bus_route_1 = bus_shared_route + [
    [37.452672, 126.950314],
    [37.450781, 126.949735],
    [37.449674, 126.949853],
    [37.447646, 126.949177],
    [37.447067, 126.951280],
    [37.447799, 126.952020],
    [37.450346, 126.951966]] # 301

bus_route_2 = bus_shared_route + [
    [37.453834, 126.951684]
]

bus_route_2.reverse()

routes = [
    bus_route_1,
    bus_route_2
]

def add_bus_route(map, color, route):  
    for bus_location in route:
        map += Marker(
            location=bus_location, 
            icon=Icon(icon_url='http://getdrawings.com/free-icon/bus-stop-icon-65.png',
                      draggable=False,
                      icon_size=[38, 38], 
                      icon_anchor=[19,38]))

    map += Polygon(
            locations=route,
            color=color,
            opacity=0.3,
            fill_opacity=0.0)
    
snu_position = (37.4556699,126.9533264)
m = Map(center=snu_position, zoom=15, basemap=basemaps.Esri.WorldStreetMap)

add_bus_route(m, "red", routes[0])
add_bus_route(m, "blue", routes[1])

sc = Sidecar(title='GPSS Test Environment')

with sc:
    display(m)
    

user_markers = []
bus_markers = []
group_layers = {}
    
# Target server (Local : 'http://localhost:8080/')
local_host = 'http://localhost:8080/'
target_server = 'http://54.180.101.171:8080/'

map_bound = [[37.447068, 126.947195],[37.466040, 126.956438]]
num_clients = 40
num_bus_per_route = 1

for i in range(num_clients):
    user_markers.append(Marker(
        location=m.center, 
        icon=icon_dict[test_client.UserStatus.NON_GROUP_MEMBER], 
        draggable=False))
    m += user_markers[i]

for i in range(num_bus_per_route * len(routes)):
    bus_markers.append(Marker(
        location=m.center, 
        icon=Icon(icon_url='https://ya-webdesign.com/images/bus-icon-png-5.png', icon_size=[60, 35], icon_anchor=[30, 17.5]), 
        draggable=False))
    m += bus_markers[i]
    
def client_callback(clients, buses):
    group_dict = {}
    for i in range(num_clients):
        if clients[i].position_from_server is not None:
            user_markers[i].location = list(clients[i].position_from_server)
        user_markers[i].icon = icon_dict[clients[i].status]
        if user_markers[i].popup is None:
            user_markers[i].popup = Text()
        user_markers[i].popup.value = 'GPS request count : ' + str(clients[i].gps_request_count)
        user_markers[i].title = 'GPS request count : ' + str(clients[i].gps_request_count)
        if clients[i].group_id is not None and clients[i].status is not test_client.UserStatus.NON_GROUP_MEMBER:
            group_dict.setdefault(clients[i].group_id, []).append(i)
    
    # Remove group layer if it's gone
    for group_id, layer in group_layers.items():
        if group_id not in group_dict:
            m.remove_layer(layer)
    
    # Add group layer if there's a new one
    for group_id, id_list in group_dict.items():
        sw = (90,180)
        ne = (0,0)
        
        for i in id_list:
            if clients[i].position_from_server is not None:
                sw = np.minimum(sw, clients[i].position_from_server)
                ne = np.maximum(ne, clients[i].position_from_server)
                
        if group_id in group_layers:
            m.remove_layer(group_layers[group_id])
            
        group_layers[group_id]= Rectangle(bounds=(list(sw), list(ne)), color=ColorHash(group_id).hex)
        m.add_layer(group_layers[group_id])
    
    for i in range(len(buses)):
        print(buses[i].position)
        bus_markers[i].location = list(buses[i].position)
    
try:
    loop = asyncio.get_running_loop()
except RuntimeError:   
    loop = asyncio.get_event_loop()

task = asyncio.create_task(test_client.execute(loop, test_client.PositionUpdateTest, num_clients, target_server, map_bound, num_bus_per_route, routes, client_callback))


In [14]:
print("Start task")

In [14]:
# You must cancel existing task here to re-run a script above.
# Resetting a event loop is not going to work properly 
# since it's shared with a IPython kernel
task.cancel()
sc.close()